<a href="https://colab.research.google.com/github/yenchunR/NLP_HW5/blob/master/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip3 install opencc-python-reimplemented

     |████████████████████████████████| 484 kB 7.4 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486150 sha256=ebb4876a42fd7ff42938bceb810196112443af2b40099151ac0b5642cfa639d3
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [49]:
!pip3 install zhon

     |████████████████████████████████| 99 kB 5.3 MB/s 
  Created wheel for zhon: filename=zhon-1.1.5-py3-none-any.whl size=84322 sha256=653c45e2c1a0afd9ea5ede74175345dde3306eb91a6b0131d524a20891cb4b16
  Stored in directory: /root/.cache/pip/wheels/d0/56/17/2675c4c7413a72bf173062e8d0a16503e3b2607745aa84988d
Successfully built zhon


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#數據處理

## 分割英文數據和中文數據

In [46]:
# coding=utf-8
from opencc import OpenCC
import json
import re

trainPath = '/content/drive/My Drive/translation2019zh/translation2019zh_train.json'

# Initial
cc = OpenCC('s2t')

en_data = []
ch_data = []
index = 0

with open(trainPath, 'r') as f:
    for line in f.readlines():
        data = json.loads(line)
        en_data.append(data['english'])
        ch_data.append('\t'+ cc.convert(data['chinese']) + '\n')
        if index > 50:
            break
        index += 1
print(en_data[:10])
print(ch_data[:10])

['For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.', 'He calls the Green Book, his book of teachings, “the new gospel.', 'And the light breeze moves me to caress her long ear', 'They have the blood of martyrs is the White to flow …', "Finally, the Lakers head to the Motor City to take on a Pistons team that currently owns the Eastern Conference's second best record (1/31). L.", '"The perfect match—my father loves names and Jackie loves money, " sneered Alexander at the wedding. Neither he nor Christina ever got along with their stepmother17.', 'In 2006, Walmart was charged with racism when its recommendation engine paired Planet of the Apes with a documentary about Martin Luther King.', 'The matte as main copper phase in the cleaning. slag was deter- mined by electron probe microscopic analysis.', 'Have you shined your shoes?', 'The Tanning Matrix can be formed by resorcinol and oxazolidine E, and the reactioncharateristics 

## 分別生成中英文字典

In [50]:
import string
from zhon.hanzi import punctuation

en_vocab = set(''.join(en_data))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

ch_vocab = set(''.join(ch_data))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print('\n英文字典:\n', en2id)
print('\n中文字典共計\n:', ch2id)


英文字典:
 ['I', 'S', 'j', 'w', ')', '9', 'i', ':', 'k', '/', 'q', 'r', 'U', 'v', 'D', 'x', 'h', 'P', 'O', '8', 'R', 'p', 'W', 'E', 'f', '3', 'A', 'C', '.', ',', 'l', '2', 'a', '“', 'N', '&', '…', "'", '5', '0', 't', 'L', '—', 'G', 'o', 'u', 'c', 'F', '1', 'g', 'b', 'B', 'z', 'T', '6', 'Y', 'y', 's', 'J', 'M', '7', '-', '?', 'm', ' ', 'H', 'e', 'n', '"', 'd', 'K', '(']

中文字典共計
: {'I': 0, '覽': 1, '求': 2, '鋼': 3, '須': 4, '運': 5, '功': 6, '物': 7, '邊': 8, '快': 9, '襪': 10, '行': 11, '必': 12, '律': 13, '碼': 14, '艾': 15, '內': 16, '腳': 17, '拿': 18, '博': 19, '粒': 20, '買': 21, '狗': 22, '杯': 23, '賺': 24, '表': 25, '…': 26, '合': 27, '酸': 28, '迷': 29, '比': 30, '似': 31, '音': 32, '水': 33, '加': 34, '協': 35, '貧': 36, '究': 37, '份': 38, '棄': 39, '出': 40, '探': 41, '至': 42, '失': 43, '麗': 44, '段': 45, '（': 46, '昨': 47, '見': 48, '應': 49, '麥': 50, '羣': 51, '爲': 52, '習': 53, '己': 54, '鞣': 55, '瀏': 56, '喜': 57, '諷': 58, 'E': 59, '、': 60, '幾': 61, '亟': 62, '願': 63, '銘': 64, '將': 65, '載': 66, '增': 67, '正': 68, '：': 69, 

## 轉換輸入數據格式

In [37]:
# 利用字典，映射數據
en_num_data = [[en2id[en] for en in line ] for line in en_data]
ch_num_data = [[ch2id[ch] for ch in line] for line in ch_data]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in ch_data]

print('char:', en_data[1])
print('index:', en_num_data[1])

char: He calls the Green Book, his book of teachings, “the new gospel.
index: [65, 66, 64, 46, 32, 30, 30, 57, 64, 40, 16, 66, 64, 43, 11, 66, 66, 67, 64, 51, 44, 44, 8, 29, 64, 16, 6, 57, 64, 50, 44, 44, 8, 64, 44, 24, 64, 40, 66, 32, 46, 16, 6, 67, 49, 57, 29, 64, 33, 40, 16, 66, 64, 67, 66, 3, 64, 49, 44, 57, 21, 66, 30, 28]


## 將訓練數據進行onehot編碼

In [38]:
import numpy as np

# 獲取輸入輸出端的最大長度
max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.
    for t, j in enumerate(de_num_data[i]):
        decoder_target_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

max encoder length: 197
max decoder length: 70
index data:
 [65, 66, 64, 46, 32, 30, 30, 57, 64, 40, 16, 66, 64, 43, 11, 66, 66, 67, 64, 51, 44, 44, 8, 29, 64, 16, 6, 57, 64, 50, 44, 44, 8, 64, 44, 24, 64, 40, 66, 32, 46, 16, 6, 67, 49, 57, 29, 64, 33, 40, 16, 66, 64, 67, 66, 3, 64, 49, 44, 57, 21, 66, 30, 28]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# 模型選擇與建模

## 參數設置

In [39]:
# =======預定義模型參數========
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.003
BATCH_SIZE = 100
EPOCHS = 200

## encoder建模

In [40]:
# ======================================keras model==================================
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.optimizers import Adam
import numpy as np

# ==============encoder=============
encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
#emb_inp = Embedding(output_dim=HIDDEN_SIZE, input_dim=EN_VOCAB_SIZE)(encoder_inputs)
encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)

## decoder建模

In [41]:
# ==============decoder=============
decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))

#emb_target = Embedding(output_dim=HIDDEN_SIZE, input_dim=CH_VOCAB_SIZE, mask_zero=True)(decoder_inputs)
lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')

decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
decoder_outputs = decoder_dense(decoder_h2)

## 訓練模型

In [42]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, None, 72)]   0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, None, 752)]  0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, None, 256),  336896      input_13[0][0]                   
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, None, 256),  1033216     input_14[0][0]                   
                                                                 lstm_8[0][1]               

# Result

## 搭建預測模型

In [43]:
# encoder模型和訓練相同
encoder_model = Model(encoder_inputs, [encoder_state_h1, encoder_state_c1, encoder_state_h2, encoder_state_c2])

# 預測模型中的decoder的初始化狀態需要傳入新的狀態
decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))

# 使用傳入的值來初始化當前模型的輸入狀態
decoder_h1, state_h1, state_c1 = lstm1(decoder_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1])
decoder_h2, state_h2, state_c2 = lstm2(decoder_h1, initial_state=[decoder_state_input_h2, decoder_state_input_c2])
decoder_outputs = decoder_dense(decoder_h2)

decoder_model = Model([decoder_inputs, decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2], 
                      [decoder_outputs, state_h1, state_c1, state_h2, state_c2])

## 利用預測模型進行翻譯

In [44]:
for k in range(3):
    test_data = encoder_input_data[k:k+1]
    h1, c1, h2, c2 = encoder_model.predict(test_data)
    target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
    target_seq[0, 0, ch2id['\t']] = 1
    outputs = []
    while True:
        output_tokens, h1, c1, h2, c2 = decoder_model.predict([target_seq, h1, c1, h2, c2])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        outputs.append(sampled_token_index)
        target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
        target_seq[0, 0, sampled_token_index] = 1
        if sampled_token_index == ch2id['\n'] or len(outputs) > 20: break
    
    print(en_data[k])
    print(''.join([id2ch[i] for i in outputs]))

For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.
然爾，，的說的的能長長長的的的的的。。。。
He calls the Green Book, his book of teachings, “the new gospel.
他他他在沒的的的的的的的的，，的的的。。。
And the light breeze moves me to caress her long ear
然爾，，的說的的能長長長的的的的的。。。。
